In [1]:
import json
from openai import OpenAI
from groq import Groq
import os
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
import pickle
from elasticsearch import Elasticsearch
import pandas as pd
import requests
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from requests.exceptions import HTTPError

import time

/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
import os
import pickle

# Define the directory containing the pickle files
directory_path = '../data/vietnamese_rag/llm_answer/'

# Initialize an empty list to store the combined data
combined_data = []

# Iterate over all files in the directory
for filename in os.listdir(directory_path):
    # Check if the file is a pickle file
    if filename.endswith('.pkl'):
        file_path = os.path.join(directory_path, filename)
        try:
            # Open and load the pickle file
            with open(file_path, 'rb') as file:
                data = pickle.load(file)
                # Ensure the loaded data is a list
                if isinstance(data, list):
                    # Extend the combined list with the data from the current pickle file
                    combined_data.extend(data)
                else:
                    print(f"Warning: {filename} does not contain a list.")
        except Exception as e:
            print(f"Error loading {filename}: {e}")

# Now combined_data contains all the dictionaries from all the pickle files
print(f"Loaded data from {len(combined_data)} dictionaries across all pickle files.")

Loaded data from 23768 dictionaries across all pickle files.


In [14]:
combined_data[0]

{'answer_llm': 'Chứng sa sút trí tuệ có thể do những căn bệnh nào gây ra?\n\nChứng sa sút trí tuệ có thể do nhiều yếu tố khác nhau gây ra, bao gồm tổn thương hoặc bệnh tật ở các phần não chịu trách nhiệm học tập, trí nhớ, ra quyết định và ngôn ngữ. Bệnh Alzheimer là nguyên nhân phổ biến nhất của chứng mất trí nhớ, chiếm 60%-80% trường hợp. Tuy nhiên, có tới 50 nguyên nhân khác gây ra chứng sa sút trí tuệ. Ngoài ra, một số vấn đề sức khỏe có thể dẫn đến các triệu chứng giống như chứng mất trí nhớ, nhiều triệu chứng trong số đó có thể điều trị được và hồi phục được.',
 'answer_orig': 'Chứng sa sút trí tuệ có thể do nhiều yếu tố khác nhau gây ra, bao gồm tổn thương hoặc bệnh tật ở các phần não chịu trách nhiệm học tập, trí nhớ, ra quyết định và ngôn ngữ [4]. Bệnh Alzheimer là nguyên nhân phổ biến nhất của chứng mất trí nhớ, chiếm 60% -80% trường hợp [4]. Tuy nhiên, có tới 50 nguyên nhân khác gây ra chứng sa sút trí tuệ [4]. Một số chứng sa sút trí tuệ không thể hồi phục và trở nên trầm tr

In [16]:
df = pd.DataFrame(combined_data)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23768 entries, 0 to 23767
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   answer_llm   23768 non-null  object
 1   answer_orig  23768 non-null  object
 2   document     23768 non-null  object
 3   question     23768 non-null  object
 4   group        23768 non-null  object
dtypes: object(5)
memory usage: 928.6+ KB


In [23]:
df.to_csv('../data/vietnamese_rag/llm_answer/llm_answer.csv', index = False)

In [28]:
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

In [24]:
def normalize_vector(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

In [25]:
def compute_similarity(record, model):
    v_llm = normalize_vector(model.encode(record['answer_llm']))
    v_orig = normalize_vector(model.encode(record['answer_orig']))
    return v_llm.dot(v_orig)

In [32]:
similarity_scores = []
results = df.to_dict(orient = 'records')
for record in tqdm(results):
    # print(record)
    sim = compute_similarity(record, model)
    similarity_scores.append(sim)

  2%|▌                      | 584/23768 [01:11<47:23,  8.15it/s]


KeyboardInterrupt: 